# LISFLOOD - CHANGE CALIBRATION PARAMETERS
***

This script is provided to quickly change the main LISFLOOD calibration parameters

**Author**: Timo Schaffhauser <br>
**Date**: 15th Of January 2026<br>
***
**Parameters LISFLOOD**<br>
| ParameterName         | MinValue | MaxValue | DefaultValue |
|-----------------------|---------:|---------:|-------------:|
| UpperZoneTimeConstant  | 0.01     | 40       | 10           |
| LowerZoneTimeConstant  | 40       | 1000     | 100          |
| GwPercValue            | 0.01     | 2        | 0.8          |
| LZThreshold            | 0        | 30       | 10           |
| b_Xinanjiang           | 0.01     | 5        | 0.5          |
| PowerPrefFlow          | 0.5      | 8        | 4            |
| SnowMeltCoef           | 2.5      | 6.5      | 4            |
| CalChanMan1            | 0.5      | 2        | 1            |
| CalChanMan2            | 0.5      | 5        | 1            |
| LakeMultiplier         | 0.5      | 2        | 1            |
| adjust_Normal_Flood    | 0.01     | 0.99     | 0.8          |
| ReservoirRnormqMult    | 0.25     | 2        | 1            |
| QSplitMult             | 0        | 20       | 2            |
| GwLoss                 | 0        | 0.5      | 0            |
| TransSub               | 0        | 0.12     | 0            |

For more information we refer to: https://ec-jrc.github.io/lisflood-code/4_annex_settings_and_options/ 

In [42]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import xml.etree.ElementTree as ET
import lisflood
import lisfloodutilities


## Inputs

In [12]:
# Settings file(s)
IN_DIR = Path("./settings/")
SETTINGS_FILE_PRE = "OSLisfloodGloFASv5calibration_v1_SouthAmerica_ChilePreRunlong_term_run.xml" 
SETTINGS_FILE = "OSLisfloodGloFASv5calibration_v1_SouthAmerica_ChileRunlong_term_run.xml"


## Read settings xml & change parameters

In [19]:
# Define settings files
SETTING_PRE = IN_DIR / SETTINGS_FILE_PRE
SETTINGS = IN_DIR / SETTINGS_FILE

# Read xml file into memory
settings_tree = ET.parse(SETTINGS)   # Read XML file and build in-memory XML tree
root = settings_tree.getroot()       # Access the root element (<lfsettings>)

#### a) General setup settings

In [ ]:
# Define setup
setup_lf = {
    # "CalendarDayStart": "02-01-1975 00:00",
    "StepStart": "02-01-1975 00:00",
    "StepEnd": "01/01/1979 00:00",
    # "timestepInit": "1",
    # "DtSec": "86400",
    # "DtSecChannel": "14400",
    "PathRoot": "/storage/schafti/Documents/01_Hydrology/01_Lisflood/00_SourceCode/01_Playground/01_Test_Usecase/create_submodel_7859/7859_testing",
    "PathOut": "$(PathRoot)/out/long_term_run",
    "PathInit": "$(PathRoot)/out/long_term_run",
    # "Gauges":"-72.675 -45.425"
}

#### b) Calibration parameters

In [33]:
# Define calibration parameters
# Best parameters:
    # b_Xinanjiang: 3.09004
    # CalChanMan1: 4.49689
    # CalChanMan3: 1
    # GwLoss: 0.0866118
    # GwPercValue: 0.878763
    # LowerZoneTimeConstant: 40
    # UpperZoneTimeConstant: 9.0402
    # LZthreshold: 5.65075
    # PowerPrefFlow: 1.51084
    # SnowMeltCoef: 5.10491
    # TransSub: 0.000135783

params_lf = {
    "b_Xinanjiang": 3,
    "CalChanMan": 3,
    "CalChanMan3": 3,
    "GwLoss": 3,
    "GwPercValue": 3,
    "LowerZoneTimeConstant": 3,
    "UpperZoneTimeConstant": 3,
    "LZThreshold": 3,
    "PowerPrefFlow": 3,
    "SnowMeltCoef": 3,
    "TransSub": 3
}


In [23]:
# Function that changes parameters silently in the xml that was read into memory
def set_textvar(
    root,
    name,
    new_value,
    *,
    must_exist=True,
):
    """
    Modify the 'value' attribute of a <textvar> element in a LISFLOOD settings file.

    Parameters
    ----------
    root : xml.etree.ElementTree.Element
        Root element of the parsed XML tree (typically <lfsettings>).
    name : str
        Value of the 'name' attribute of the <textvar> to modify
        (e.g. "ChanSMult").
    new_value : str or float or int
        New value to assign to the 'value' attribute.
    must_exist : bool, optional
        If True (default), raise an error if the parameter is not found.
        If False, silently do nothing if the parameter does not exist.

    Returns
    -------
    bool
        True if the parameter was found and modified, False otherwise.
    """

    for tv in root.iter("textvar"):
        if tv.attrib.get("name") == name:
            tv.attrib["value"] = str(new_value)
            return True

    if must_exist:
        raise KeyError(f"textvar '{name}' not found in settings file")

    return False

In [40]:
# Change setup parameters
# Example 1) manually one at a time
set_textvar(root, "StepEnd", "01/01/1979 00:00", must_exist=True)

# Example 2) Loop over setup_lf dict and apply changes
# for name, value in setup_lf.items():
#     changed = set_textvar(root, name, value, must_exist=False)
#     if not changed:
#         print(f"⚠ Parameter '{name}' not found — no change applied")


# Change calib parameters
# Example 1) manually one at a time
# set_textvar("ChanSMult", 1.25)
# set_textvar("ChanDepthTMult", 0.8)
# set_textvar("ChanBottomWMult", 1.1)

# Example 2) Loop over parameters_lf dict and apply changes
for name, value in params_lf.items():
    changed = set_textvar(root, name, value, must_exist=False)
    if not changed:
        print(f"⚠ Parameter '{name}' not found — no change applied")

# write new settings file
settings_tree.write(IN_DIR / "settings_modified.xml")